In [30]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import requests
import orjson as json

chrome_options = Options()
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--proxy-server=socks5://127.0.0.1:9999")
# chrome_options.add_argument(" --host-resolver-rules='MAP * ~NOTFOUND, EXCLUDE localhost'")
df = pd.read_csv('yahoo_tickers.csv')
with open('passed.txt') as f:
    tickers = f.read().splitlines()
with open('ticker_capm.json') as f:
    ticker_capm = json.loads(f.read())
# driver = webdriver.Chrome(options=chrome_options)

In [37]:
def capm(ticker: str) -> float:
    ticker_split = ticker.split('.')[0]
    if ticker.endswith('.TO'):
        market = 'TSX'
    elif ticker.endswith('.SA'):
        market = 'BOVESPA'
    elif ticker.endswith('.HK'):
        market = 'SEHK'
        ticker = str(int(ticker))
    elif ticker.endswith('.BO'):
        r = requests.get(f'https://finbox.com/_/api/v5/search-v2/company?q={ticker_split}').text
        r = json.loads(r)
        for i in r:
            if i['exchange'] == 'BSE':
                market, ticker_split = i['full_ticker'].split(':')
                break
        else:
            market = 'NSEI'
    elif ticker.endswith('.SZ'):
        market = 'SZSE'
    elif ticker.endswith('.SS'):
        market = 'SHSE'
    elif ticker.endswith('.MX'):
        market = 'BMV'
    elif ticker.endswith('.L'):
        if ticker_split == 'WYN':
            market = 'AIM'
        else:
            market = 'LSE'
    elif df.loc[df['Ticker'] == ticker, 'Exchange'].to_list()[0] == 'Nasdaq':
        if ticker == 'DJCO':
            market = 'NASDAQCM'
        else:
            market = 'NASDAQGS'
    elif df.loc[df['Ticker'] == ticker, 'Exchange'].to_list()[0] == 'New York Stock Exchange, Inc.':
        market = 'NYSE'
    else:
        raise Exception('ticker not matched!')
    url = f'https://finbox.com/{market}:{ticker_split}/explorer/cost_equity'
    driver.get(url)
    try:
        wacc_tag = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//li[contains(text(), 'cost of equity is')]")))
    except TimeoutException:
        print(ticker, 'timed out!')
    else:
        r = float(wacc_tag.text.split('cost of equity is ')[-1].rstrip('%.').strip())
        return r

In [34]:
ticker_capm = {'000333.SZ': 7.5, '000858.SZ': 8.4, '300760.SZ': 6.4, '600007.SS': 8.0, '600519.SS': 7.6, 'ABT': 7.1, 'AFL': 8.0, 'ALL': 7.9, 'AMD': 8.4, 'AMZN': 8.9, 'ARCB': 10.1, 'ATD.TO': 7.6, 'BABA': 12.0, 'BBSE3.SA': 9.9, 'CASY': 7.8, 'CIPLA.BO': 7.6, 'COST': 6.9, 'CTSH': 7.6, 'CTVA': 7.6, 'DJCO': 8.6, 'ECLERX.BO': 9.9, 'FCN': 7.3, 'FFH.TO': 11.3, 'FTNT': 7.5, 'GOOG': 7.9, 'GRMN': 7.9, 'GWO.TO': 9.4, 'HCLTECH.BO': 8.9, 'HDFCBANK.BO': 9.1, 'HSBA.L': 8.8, 'ICICIBANK.BO': 9.1, 'INFY.BO': 8.9, 'JJSF': 6.3, 'JPM': 8.3, 'LLOY.L': 8.5, 'LMT': 7.3, 'LULU': 7.5, 'MANH': 8.2, 'MET': 16.3, 'META': 7.9, 'MOV': 8.6, 'MSFT': 7.6, 'MTB': 7.5, 'NFLX': 8.8, 'NIITLTD.BO': 8.9, 'NOC': 8.5, 'NTES': 10.3, 'PFE': 6.1}

In [41]:
driver = webdriver.Chrome(options=chrome_options)
for ticker in tickers[66:]:
    print(ticker)
    ticker_capm[ticker] = capm(ticker)

WYN.L


In [42]:
with open('ticker_capm.json', 'w+') as f:
    f.write(json.dumps(ticker_capm).decode())